In [1]:
import os, sys, logging, importlib, math
import rasterio, affine

import geopandas as gpd
import pandas as pd

from zipfile import ZipFile
from affine import Affine
from rasterio import features
from rasterio.mask import mask
from rasterio.features import rasterize

import GEP

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
import GEP
#GEP.get_assumptions("0_0_0_0_0_0")
GEP.get_assumptions("1_1_1_1_1_0")

['Electricity demand target: Top-down low',
 'Social & commercial uses: Residential only',
 'Grid generation cost: High',
 'PV cost: Low',
 'Intermediate investment: Capped connections in 2025',
 'Rollout: Least-cost nationwide']

In [5]:
scenario = "0_0_0_0_1_0"
scenariosFolder=f'/media/gost/DATA1/GEP/Scenarios/{scenario}'
gFields = ['id','geometry','FinalElecCode2025','FinalElecCode2030', 'Pop2030']

countries = []
for x in os.listdir(scenariosFolder):
    if os.path.isdir(os.path.join(scenariosFolder, x)):
        countries.append(x[:4])


In [7]:
importlib.reload(GEP)
# Summarize electrification numbers by admin region
adm_data = '/media/gost/DATA1/GEP/OTHER_DATA/geoBoundaries-LBR-ADM2.geojson'
inA = gpd.read_file(adm_data)

country = 'lr-2'
xx_so = GEP.gep_results(country, 
                    scenariosFolder=f'/media/gost/DATA1/GEP/Scenarios/',
                    clustersFolder = '/media/gost/DATA1/GEP/Clusters',
                    scenario=scenario)
inD = xx_so.join_results()

In [53]:
inA['gep_pop'] = 0
inA['gep_elec'] = 0

for idx, row in inA.iterrows():
    sel_s = inD.loc[inD.intersects(row['geometry'])].copy()
    y = row['geometry']
    sel_s['area'] = sel_s['geometry'].apply(lambda x: y.intersection(x).area/x.area)
    sel_s['Population'] = sel_s['Population'] * sel_s['area']
    sel_s['ElecPop'] = sel_s['ElecPop'] * sel_s['area']
    
    inA.loc[idx, 'gep_pop'] = sel_s['Population'].sum()
    inA.loc[idx, 'gep_elec'] = sel_s['ElecPop'].sum()

In [54]:
inA.head()

,shapeName,Level,shapeID,shapeGroup,shapeType,geometry,gep_pop,gep_elec
0,B'hai,ADM2,LBR-ADM2-62588387B84276869,LBR,ADM2,"POLYGON ((-8.610305866999965 6.50005193100003,...",13879.631,3171.991
1,Barclayville,ADM2,LBR-ADM2-62588387B17252515,LBR,ADM2,POLYGON ((-8.395838529999935 4.650372723000032...,12443.617,2867.541
2,Beawor,ADM2,LBR-ADM2-62588387B57640420,LBR,ADM2,"POLYGON ((-9.155244357999948 5.76216753500006,...",5127.822,0.000
3,Belleh,ADM2,LBR-ADM2-62588387B61058481,LBR,ADM2,POLYGON ((-9.636307138999939 7.532652742000039...,19790.661,0.000
4,Bleebo,ADM2,LBR-ADM2-62588387B12471842,LBR,ADM2,POLYGON ((-8.013066422999941 4.727859186000046...,1848.559,0.000


In [55]:
inA['gep_pop'].sum()

4426868.4397952985

In [56]:
inA.to_file(adm_data.replace(".geojson", "_ENERGY_ACCESS.shp"))

In [47]:
y = row['geometry']
sel_s['area'] = sel_s['geometry'].apply(lambda x: y.intersection(x).area/x.area)

/home/gost/anaconda3/envs/Geog/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [48]:
sel_s

,id,Population,ElecPop,NightLight,Area,geometry,Unnamed: 0,AgriDemand,CommercialDemand,Country,...,MinimumOverallLCOE2030,MinimumOverallCode2030,InvestmentCost2030,ElecStatusIn2030,InvestmentCapita2030,FinalElecCode2030,NewCapacity2030,GridPop2030,OffGridPop2030,area
75,61140,8.416,0.000,0.000,0.009,"POLYGON ((-9.270702760733043 7.1587500225078, ...",59231,0,0,Liberia,...,0.694,3.000,27.253,1,38.636,3.000,0.003,0.000,10.835,1.000
1591,50599,92.572,0.000,0.000,0.071,POLYGON ((-9.286519427526532 7.230425023083373...,49044,0,0,Liberia,...,0.353,3.000,31592.188,1,265.079,3.000,7.068,0.000,119.180,1.000
1643,55766,143.066,0.000,0.000,0.043,POLYGON ((-9.272267015399891 7.248741689897286...,53980,0,0,Liberia,...,0.694,3.000,923.864,1,77.042,3.000,0.105,0.000,184.188,1.000
1769,57433,100.988,0.000,0.000,0.035,POLYGON ((-9.285702760853182 7.265416690031139...,55602,0,0,Liberia,...,0.695,3.000,653.216,1,77.169,3.000,0.074,0.000,130.015,1.000
3232,54760,185.144,0.000,0.000,0.079,POLYGON ((-9.281536094153143 7.297916690292159...,52988,0,0,Liberia,...,0.694,3.000,1196.128,1,77.077,3.000,0.136,0.000,238.361,1.000
3929,56733,8.416,0.000,0.000,0.009,POLYGON ((-9.623202763556295 7.224583356369861...,54918,0,0,Liberia,...,0.694,3.000,27.223,1,38.593,3.000,0.003,0.000,10.835,1.000
3932,56451,8.416,0.000,0.000,0.009,POLYGON ((-9.641536097036465 7.181250022688503...,54638,0,0,Liberia,...,0.696,3.000,27.329,1,38.742,3.000,0.003,0.000,10.835,1.000
3938,56262,16.831,0.000,0.000,0.009,POLYGON ((-9.638202763676437 7.182083356028527...,54463,0,0,Liberia,...,0.696,3.000,54.657,1,38.742,3.000,0.006,0.000,21.669,1.000
4013,59476,8.416,0.000,0.000,0.009,POLYGON ((-9.660702763856644 7.199583356169077...,57596,0,0,Liberia,...,0.693,3.000,27.179,1,38.530,3.000,0.003,0.000,10.835,1.000
4018,59190,8.416,0.000,0.000,0.009,POLYGON ((-9.647369430416521 7.207916689569339...,57315,0,0,Liberia,...,0.693,3.000,27.179,1,38.530,3.000,0.003,0.000,10.835,1.000


In [46]:
y.intersection(sel_s['geometry'].iloc[0]).area

7.224403000584603e-07

# Join scenario results to cluster shapefile

In [8]:
country = 'cm-1'
xx_so = gep_results(country, 
                    scenariosFolder=scenariosFolder,
                    clustersFolder = '/media/gost/DATA1/GEP/Clusters',
                    scenario=scenario)
inD = xx_so.join_results()
inD.head()

FileNotFoundError: [Errno 2] No such file or directory: '/media/gost/DATA1/GEP/Scenarios/0_0_0_0_1_0/0_0_0_0_1_0'

In [ ]:
inD.to_file('/media/gost/DATA1/GEP/Clusters/GEP_clusters_%s.shp' % scenario)

In [ ]:
tempD = inD.loc[inD['Pop'] > 10,['FinalElecCode2025','FinalElecCode2030','Population','geometry']]
tempD.shape

allRes = {}
cnt = 0
for country in countries:
    if not country in allRes.keys():
        print(f'Processing {country}')
        xx_so = gep_results(country, 
                    scenariosFolder=f'/media/gost/DATA1/GEP/Scenario_{scenario}',
                    clustersFolder = '/media/gost/DATA1/GEP/Clusters',
                    scenario=scenario)
        inD = xx_so.join_results()
        # Identify rows where the electrified source changes between 2025 and 2030
        changing_source = (inD['FinalElecCode2025'] > inD['FinalElecCode2030']) & (inD['FinalElecCode2025'] != 99)
        # For those changed rows, set the adjusted 2030 value = 2025 value
        inD['adjust_2030'] = inD['FinalElecCode2030']
        inD['ElecPop2030'] = inD['Pop2030'] - inD['ElecPop']
        inD.loc[changing_source,'adjust_2030'] = inD.loc[changing_source, 'FinalElecCode2025']
        allRes[country] = [inD.loc[:,['adjust_2030','ElecPop2030','ElecPop','Pop']].groupby('adjust_2030').sum(),
                           inD.loc[:,['FinalElecCode2030','ElecPop2030','ElecPop','Pop']].groupby('FinalElecCode2030').sum()]
    else:
        print(f'Already processed {country}')
    '''cnt += 1
    if cnt > 1:
        break'''

In [ ]:
allRes = {}
cnt = 0
for country in countries:
    if not country in allRes.keys():
        print(f'Processing {country}')
        xx_so = gep_results(country, 
                    scenariosFolder=f'/media/gost/DATA1/GEP/Scenario_{scenario}',
                    clustersFolder = '/media/gost/DATA1/GEP/Clusters',
                    scenario=scenario)
        inD = xx_so.join_results()
        # Identify rows where the electrified source changes between 2025 and 2030
        changing_source = (inD['FinalElecCode2025'] > inD['FinalElecCode2030']) & (inD['FinalElecCode2025'] != 99)
        # For those changed rows, set the adjusted 2030 value = 2025 value
        inD['adjust_2030'] = inD['FinalElecCode2030']
        inD['ElecPop2030'] = inD['Pop2030'] - inD['ElecPop']
        inD.loc[changing_source,'adjust_2030'] = inD.loc[changing_source, 'FinalElecCode2025']
        allRes[country] = [inD.loc[:,['adjust_2030','ElecPop2030','ElecPop','Pop','NewConnections2030','NewConnections2025']].groupby('adjust_2030').sum(),
                           inD.loc[:,['FinalElecCode2030','ElecPop2030','ElecPop','Pop','NewConnections2030','NewConnections2025']].groupby('FinalElecCode2030').sum()]
    else:
        print(f'Already processed {country}')
    '''cnt += 1
    if cnt > 1:
        break'''